# 나이브베이즈

- [Amazon product data](http://jmcauley.ucsd.edu/data/amazon/)의 baby 데이터 사용
- 아마존 상품에 대한 리뷰의 긍정과 부정 분석

`scikit-learn`의 `naive_bayes` 라이브러리


- `MultinomialNB`: 멀티노미얼 나이브베이즈
- `GaussianNB`: 가우시안 나이브베이즈

`scikit-learn`의 `feature_extraction.text` 라이브러리: 어휘(특성) 추출
- `CountVectorizer`: 단어들의 출현 빈도(frequency)로 벡터화

In [61]:
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

리뷰의 overall 항목이 4보다 작으면 부정, 4보다 크면 긍정으로 분류하고 리뷰 문장이 들어있는 reviewText 컬럼만 가져온다. 또한, 데이터가 매우 크므로 부정, 긍정 각각 1000개씩만 가져온 데이터로 실습한다. colab으로 데이터를 가공해준 뒤 reviews.csv 파일로 저장하여 사용하였다.

In [62]:
review = pd.read_csv('../input/reviews/reviews.csv')
review = review.drop('Unnamed: 0', axis=1)
review.head()

,review,label
0,This just did not do the trick for me. I'm 5'2...,neg
1,"I don't consider myself a wipe this, wipe that...",neg
2,I was so excited for this sheet because I LOVE...,neg
3,"Ugh, where do I start? The thing is complete c...",neg
4,Overall I'm happy with this product. It delive...,neg


### 1. CounterVectorizer

나이브 베이즈 모델을 활용하기 위해서는 먼저 데이터를 scikit-learn이 활용할 수 있는 형태로 transform 해주어야한다. 그러기 위해서 CounterVectorizer 객체를 생성하고 .fit() 메서드로 단어들을 학습시킨다.

In [90]:
data = review['review'].values.astype('U')
data

array(["This just did not do the trick for me. I'm 5'2&#34;, so I thought the mini snoogle would be better for me. I really didn't like anything about it. The top part is uncomfortable to use as a pillow, which is one way it is shown for usage. I have tried bending it all sorts of different ways, and just none are comfortable. It doesn't contour to my body well at all. I wound up getting a regular body pillow and am happier with that.",
       "I don't consider myself a wipe this, wipe that don't touch that kind of mom but these cart covers just make sense.Here is when it REALLY clicked for me.. I had the little boy I was taking care of in the Brunos cart and he had an accident it went everywhere...all over the buggie seat. I spent 5 minutes wiping down the buggy in the parking lot and that was still not enough to clean it and honestly 5 more minutes then anyone else would have spent...When I had kids...I promptly purchased one of these.. I found that to make it versitile...and fit mor

**💫 리스트로 받아서 fitting 할 때 에러**

`data = review['review'].values.tolist()`로 data를 리스트 형태로 받아서 fitting 하려고 하면 `np.nan is an invalid document, expected byte or unicode string`이라는 에러가 발생한다.



이는 Unicode 변환을 해야 vectorizer fitting을 수행할 수 있기 때문으로, Unicode 변환을 해주면 정상적으로 동작한다.

In [91]:
vectorizer = CountVectorizer()
vectorizer.fit(data)

CountVectorizer()

### 2. transform

문자열 목록을 가져와 미리 학습해놓은 사전을 기반으로 어휘의 빈도를 세주는 메서드이다.<br>
학습시킨 vectorizer를 .transform() 메서드로 문자열의 배열을 받아서 학습된 단어들의 빈도수로 변환하자. counts에는 각 단어가 등장한 빈도수가 저장된다.
- counts의 index: 학습 단어들
- counts의 값: 해당 단어의 빈도수

In [111]:
vectorizer.vocabulary_

{'this': 7974,
 'just': 4234,
 'did': 2295,
 'not': 5191,
 'do': 2445,
 'the': 7912,
 'trick': 8216,
 'for': 3209,
 'me': 4771,
 '34': 105,
 'so': 7164,
 'thought': 7982,
 'mini': 4885,
 'snoogle': 7143,
 'would': 8893,
 'be': 819,
 'better': 910,
 'really': 6212,
 'didn': 2296,
 'like': 4483,
 'anything': 525,
 'about': 226,
 'it': 4155,
 'top': 8112,
 'part': 5517,
 'is': 4141,
 'uncomfortable': 8328,
 'to': 8069,
 'use': 8453,
 'as': 599,
 'pillow': 5663,
 'which': 8757,
 'one': 5297,
 'way': 8681,
 'shown': 6924,
 'usage': 8452,
 'have': 3680,
 'tried': 8219,
 'bending': 899,
 'all': 419,
 'sorts': 7245,
 'of': 5266,
 'different': 2306,
 'ways': 8682,
 'and': 484,
 'none': 5173,
 'are': 574,
 'comfortable': 1676,
 'doesn': 2454,
 'contour': 1834,
 'my': 5061,
 'body': 1006,
 'well': 8725,
 'at': 634,
 'wound': 8896,
 'up': 8422,
 'getting': 3408,
 'regular': 6318,
 'am': 457,
 'happier': 3647,
 'with': 8829,
 'that': 7908,
 'don': 2466,
 'consider': 1782,
 'myself': 5063,
 'wipe': 

In [109]:
counts = vectorizer.transform(data)
print(counts.shape)

(2000, 8986)


### 3. 나이브베이즈 분류기 학습

데이터 포인트 배열인 counts 객체와 각 데이터의 라벨을 전달하여 나이브베이즈를 학습시킨다. counts 객체를 만들 때 리뷰 데이터가 부정 1000개, 긍정 1000개 순으로 들어갔기 때문에 앞의 1000개는 0으로, 뒤의 1000개는 1로 라벨링해주면 된다.

In [110]:
classifier = MultinomialNB()

labels = [0] * 1000 + [1] * 1000
classifier.fit(counts, labels)

MultinomialNB()

### 4. 나이브베이즈 분류 예측

- `.predict()`: 임의의 데이터 포인트 배열을 전달하여 클래스 예측
- `.predict_proba()` : 주어진 데이터가 특정 클래스에 속할 확률

In [113]:
print(classifier.predict(counts))

[0 0 0 ... 1 1 1]


In [114]:
print(classifier.predict_proba(counts))

[[9.99999815e-01 1.84832697e-07]
 [1.00000000e+00 1.21459481e-10]
 [9.49704189e-01 5.02958114e-02]
 ...
 [7.41652779e-09 9.99999993e-01]
 [2.51164650e-07 9.99999749e-01]
 [3.55657697e-02 9.64434230e-01]]


임의의 리뷰가 주어졌을 때 해당 리뷰가 긍정적인지 부정적인지 분류해보자.<br>
These are great라는 리뷰는 약 0.76의 확률로 1 (긍정)으로 예측되었다.

In [115]:
review = "These are great"
review_counts = vectorizer.transform([review])

print(classifier.predict(review_counts))
print(classifier.predict_proba(review_counts))

[1]
[[0.23963667 0.76036333]]


---

### 1. 데이터의 빈도수 저장

긍정, 부정 데이터의 빈도수를 저장하는 Counter 객체 생성하기

In [94]:
# 긍정 리뷰에 포함된 단어의 빈도수 저장
pos_list = review[review['label']=='pos']['review'].values.tolist()

pos_words = []
for sentence in pos_list:
    pos_words.extend(str(sentence).split(' '))
pos_counter = Counter(pos_words)
pos_counter

Counter({'This': 283,
         'is': 1541,
         'a': 2079,
         'nice,': 7,
         'firm,': 1,
         'well-covered': 1,
         'mattress.': 6,
         '': 1784,
         'Unfortunately,': 2,
         'it': 1710,
         'just': 358,
         'little': 266,
         'too': 149,
         'small': 88,
         '(width-wise)': 1,
         'for': 1246,
         'theGraco': 2,
         'Modern': 1,
         'Pack': 10,
         "'n": 10,
         'Play': 6,
         'Playard': 2,
         'with': 815,
         'Bassinet': 1,
         '&': 41,
         'Changer,': 1,
         'Zurich.': 1,
         'My': 251,
         'husband,': 2,
         'who': 67,
         'will': 247,
         'usually': 15,
         'try': 39,
         'anything,': 1,
         "isn't": 44,
         'comfortable': 49,
         'the': 4193,
         'gap': 3,
         'our': 280,
         'baby': 448,
         'on': 741,
         'way.': 9,
         'Yes,': 10,
         'we': 407,
         'could': 114,


In [95]:
# 부정 리뷰에 포함된 단어의 빈도수 저장
neg_list = review[review['label']=='neg']['review'].values.tolist()

neg_words = []
for sentence in neg_list:
    neg_words.extend(str(sentence).split(' '))
neg_counter = Counter(neg_words)
neg_counter

Counter({'This': 236,
         'just': 448,
         'did': 135,
         'not': 810,
         'do': 219,
         'the': 5096,
         'trick': 4,
         'for': 1269,
         'me.': 46,
         "I'm": 180,
         "5'2&#34;,": 1,
         'so': 669,
         'I': 2945,
         'thought': 86,
         'mini': 10,
         'snoogle': 3,
         'would': 442,
         'be': 527,
         'better': 140,
         'really': 309,
         "didn't": 138,
         'like': 449,
         'anything': 45,
         'about': 200,
         'it.': 233,
         'The': 602,
         'top': 78,
         'part': 56,
         'is': 1690,
         'uncomfortable': 17,
         'to': 2989,
         'use': 294,
         'as': 496,
         'a': 2372,
         'pillow,': 4,
         'which': 182,
         'one': 434,
         'way': 126,
         'it': 2278,
         'shown': 4,
         'usage.': 2,
         'have': 813,
         'tried': 90,
         'bending': 3,
         'all': 274,
         'sort

### 2. 긍정 부정 확률 계산

**P(positive), P(negative) 계산하기**

긍정, 부정 리뷰를 1000개씩 가져왔으므로 긍정, 부정의 확률은 각각 0.5 이다.<br>

In [ ]:
percent_pos, percent_neg = 0.5, 0.5

### 3. 긍정일 경우 리뷰의 조건부 확률 계산

**P(review | positive) 계산하기**

해당 리뷰의 각 단어들이 모두 독립이라는 가정으로 각 단어들이 긍정 리뷰에서 나타날 확률을 구해서 곱해준다<br>
These are great라는 리뷰가 있을 때 `P(These|positive) x P(are|positive) x P(great|positive)`

In [107]:
review = "These are great"

# 전체 긍정, 부정 단어의 수
total_pos = sum(pos_counter.values())
total_neg = sum(neg_counter.values())

review_given_pos = 1
review_given_neg = 1

# smoothing: 단어가 기존의 리뷰에 존재하지 않는 경우 확률이 0이 되므로 1을 더해준다
for word in review.split(' '):
    review_given_pos *= (pos_counter[word] + 1) / (total_pos + len(pos_counter))
    review_given_neg *= (neg_counter[word] + 1) / (total_neg + len(neg_counter))

print(review_given_pos)
print(review_given_neg)

1.0870321601593177e-08
3.2438938170706177e-09


### 4. 분류하기

**P(positive | review) 와 P(negative | review) 계산**

임의의 리뷰가 주어졌을 때 해당 리뷰가 긍정적인지 부정적인지 분류하기<br>
최종 확률은 pos가 더 크므로 These are great라는 리뷰는 긍정으로 분류될 것이다

In [108]:
pos = review_given_pos * percent_pos
neg = review_given_neg * percent_neg
print(pos)
print(neg)

5.4351608007965885e-09
1.6219469085353089e-09
